In [1]:
import os
import pandas as pd
import numpy as np
from django.db import transaction, models
from django.conf import settings
from intranet.academic.models import (
    SchoolYear,
    AcademicYear, 
    CourseType, 
    CourseGroup, 
    ManagementUnit,
    Curriculum,
    Concentration,
    Course,
    CourseRequisite,
    Student
)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [16]:
def get_score_status(alpha_score):
    if alpha_score not in ['A', 'B', 'C', 'D']:
        return '-'
    status = {
        'A': 'lulus',
        'B': 'lulus',
        'C': 'lulus',
        'D': 'mengulang',
        'E': 'mengulang'
    }
    return status[alpha_score]


def get_score_classname(alpha_score):
    print(alpha_score)
    if alpha_score not in ['A', 'B', 'C', 'D']:
        return ''
    status = {
        'A': 'success',
        'B': 'success',
        'C': 'warning',
        'D': 'danger',
        'E': 'danger'
    }
    return status[alpha_score]

def get_all_scores(self, semester=None):
    courses = self.curriculum.curriculum_courses.all()
    if semester:
        courses = courses.filter(semester_number=semester)
    scores = courses.annotate(
        alphabetic=self.course_max_score_subquery('alphabetic'),
        numeric=self.course_max_score_subquery('numeric'),
        sks=self.course_max_score_subquery('sks'),
        point=self.course_max_score_subquery('point'),
        status=models.Value(get_score_status('alphabetic'), output_field=models.CharField()),
        classname=get_score_classname(models.F('alphabetic')),
    )
    return scores

def get_scores_summary(scores):
    summary = scores.aggregate(
        course_remedy=models.Count('id', filter=models.Q(numeric__gt='C')),
        course_graduate=models.Count('id', filter=models.Q(numeric__lte='C')),
        course_total=models.Count('id', filter=models.Q(alphabetic__isnull=False)),
        sks_remedy=models.Sum('sks', filter=models.Q(alphabetic__gt='C')),
        sks_graduate=models.Sum('sks', filter=models.Q(alphabetic__lte='C')),
        sks_total=models.Sum('sks', filter=models.Q(alphabetic__isnull=False)),
        total_point=models.Sum('point', filter=models.Q(alphabetic__isnull=False)),
    )
    summary['ips'] = summary['total_point'] / summary['sks_total']
    return summary

student = Student.objects.first()
scores = get_all_scores(student, semester=5)
summary = get_scores_summary(scores)
tab1 = pd.DataFrame(data=scores.values('id', 'name', 'sks', 'numeric', 'alphabetic', 'point', 'status', 'classname'))
print(tab1)
tab2 = pd.DataFrame(data=summary, index=[0])
tab2

F(alphabetic)


TypeError: QuerySet.annotate() received non-expression(s): .